### 编写策略逻辑

一般来说包含三部分：策略初始设置、定义选股逻辑、定义买卖逻辑。详细说明参见：*帮助-用户手册-逐bar回测说明*

In [ ]:

"""
选择ep因子排名前50只股票，每月初调整股票池

10日均线上穿50日均线，买入持有；
10日均线跌破50日均线，卖出清仓。
"""

import numpy as np

def initialize(context):
    """
    初始
    """
    
    # 设置初始金额
    context.initial_capital = 10000000.
    
    # 订阅行情, 分别指定 bar周期 与 股票列表
    context.freq = '1D'
    # 订阅所有中证500成分股行情数据
    context.set_scope('000905.SH')
    
    # 设定参照基准
    context.benchmark = '000905.SH'
    # 设定回测起始时间
    context.start_date = '20170104'
    context.end_date = '20171231'
    
    # 设置交易费率
    context.set_cost(ratio=0.001)
    
    # 设定股票池刷新频率
    context.universe_freq = 'monthly'
    

    
def build_universe(context):
    """
    择股逻辑
    """
    # 先读取当前的中证500成分股
    zz500 = context.get_components('000905.SH') 
    
    # 获取ep因子
    ep = context.factors['EP_TTM']
    
    # 从因子中选出排名前50的股票
    my_basket = ep.top(50, codes=zz500)
    context.universe.set(my_basket)
    

def handle_data(context, data):
    """
    策略逻辑
    """
    
    account = context.account
    # 需要同时遍历自选池中以及持仓中的股票
    codes = sorted(set(context.universe + context.holdings))
    
    for s in codes:
        close_10 = data[s].close(1,10) # 取前10日收盘价
        close_50 = data[s].close(1, 50) # 取前50日收盘价
        ma_10 = close_10.mean()
        ma_50 = close_50.mean()

        # 10日均线上穿50日均线买入
        if account[s].position == 0 and ma_10 > ma_50:
            # 全仓买入
            context.order_target_percent(s, 0.02)
            context.log('买入{}, 当前持仓{}手'.format(s, account[s].position))

        # 10日均线跌破50日均线卖出
        if account[s].position > 0 and ma_10 < ma_50:
            # 清仓
            context.order_target_value(s, 0)
            context.log('卖出{}, 当前持仓{}手'.format(s, account[s].position))
    
        
        # 卖出在持仓中但不在股票池中的股票
        if s in context.holdings:
            if s not in context.universe: 
                context.order_target_value(s, 0)
                

### 运行策略

In [ ]:
from dwtrader.core.engine import Engine
# 默认会加载当前文件内的策略
engine = Engine()
engine.run()

### 显示收益情况

In [ ]:
engine.plot(verbose=True)

### 查看交易记录

In [ ]:
engine.get_trades()